In [ ]:
!pip install deep_translator langdetect -qq -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 42.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
!pip install gTTS -qq -U

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -qq -U

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from deep_translator import GoogleTranslator
from langdetect import detect
from gtts import gTTS
# import os
import tempfile
import IPython.display as ipd
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
from transformers import TextStreamer
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "shrimantasatpati/llama3.1_finetuned_on_BI55_Medtext",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # torch_dtype=torch.float16
        # dtype = torch.float16
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [ ]:
!rm -rf /content/huggingface_tokenizers_cache

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from unsloth import FastLanguageModel

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "shrimantasatpati/llama3.1_finetuned_on_BI55_Medtext",
    max_seq_length = 8192,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [ ]:
# def apply_prompt_template(input_text):
#     template = """You are a highly knowledgeable medical AI assistant.\nYour task is to provide accurate and helpful information about medical conditions, symptoms, and general health advice.
#     \nPlease answer the following question or respond to the statement:

#     Question/Statement: {input}

#     Please provide a detailed and informative response, including relevant medical information, potential symptoms, common treatments, and when to seek professional medical advice.
#     Always emphasize the importance of consulting with a healthcare professional for personalized medical advice and diagnosis.

#     Response:"""

#     return template.format(input=input_text)
def apply_prompt_template(input_text):
    template = f"""You are an experienced and compassionate medical doctor. A patient has come to you with the following concern or symptoms:

{input_text}

Please provide a response in the following format:

1. Possible Diagnosis:
   [Provide a brief list of potential diagnoses based on the symptoms described]

2. Symptoms Analysis:
   [Describe the symptoms in more detail, explaining their significance]

3. Recommended Tests or Examinations:
   [Suggest any tests or examinations that might be necessary to confirm the diagnosis]

4. Treatment Recommendations:
   [Provide a detailed treatment plan, including medications (if applicable), lifestyle changes, and any other relevant advice]

5. Follow-up Instructions:
   [Explain when and how the patient should follow up, including any warning signs to watch for]

6. Preventive Measures:
   [Offer advice on how to prevent similar issues in the future or manage ongoing conditions]

Remember to use clear, compassionate language that a patient can understand. Always emphasize the importance of seeking professional medical advice for accurate diagnosis and treatment.

Response:
"""
    return template

In [ ]:
def convert_to_english(text):
    # Detect the source language
    try:
        source_lang = detect(text)
    except:
        raise ValueError("Language detection failed.")

    if source_lang == 'en':
        return text, source_lang
    # Translate to English
    try:
        translated_text = GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        raise ValueError(f"Translation failed: {str(e)}")
    return translated_text, source_lang

def finetune_model(input_text):
    print("Input text is:", input_text)
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    prompt = apply_prompt_template(input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    print("Generating response...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.1,
            # top_k=50,
            # top_p=0.95,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print("Full generated text:", generated_text)
    return generated_text

def medical_llm(original_query, initial_response):
  model_id = "aaditya/OpenBioLLM-Llama3-70B"

  pipeline = transformers.pipeline(
      "text-generation",
      model=model_id,
      model_kwargs={"torch_dtype": torch.bfloat16},
      device="auto",
  )

  messages = [
      {"role": "system", "content": "You are OpenBioLLM, an expert from the healthcare and biomedical domain with extensive medical knowledge and practical experience. You were developed by Saama AI Labs. Your task is to verify and enhance the given medical advice, ensuring it's accurate, comprehensive, and safe. Provide additional insights or corrections if necessary."},
      {"role": "user", "content": f"Original query: {original_query}\n\nInitial response: {initial_response}\n\nPlease verify this response, add any missing important information, and provide a final, comprehensive answer."}
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=512,  # Increased to allow for a more comprehensive response
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.1,  # Lowered for more focused responses
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

def rag_processing(query):
    pass

def web_search(query):
    pass

def generate_final_response(rag_output, web_search_output):
    # Combine RAG and web search outputs
    pass

def translate_to_original(text, target_lang):
    # Translate back to the original language
    try:
        translated_text = GoogleTranslator(source='en', target=target_lang).translate(text)
    except Exception as e:
        raise ValueError(f"Translation failed: {str(e)}")
    return translated_text

def convert_to_audio(text, lang):
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
            temp_filename = fp.name
        tts.save(temp_filename)
        # Display the audio player in the notebook
        audio = ipd.Audio(temp_filename)
        ipd.display(audio)
        print(f"Created audio file: {temp_filename}")
        return temp_filename

    except Exception as e:
        raise ValueError(f"Audio conversion failed: {str(e)}")

# input_text = "Quels sont les symptômes du diabète?"
input_text = input("Enter the text")
english_text, detected_lang = convert_to_english(input_text)
answer = finetune_model(english_text)
print(answer)
original_language_text = translate_to_original(answer, detected_lang)
audio_file = convert_to_audio(original_language_text, detected_lang)
print(f"Original ({detected_lang})\nTranslated: {english_text}")
print(f"Audio file created: {audio_file}")
# final_response, translated_response, audio_response = process_input(input_text, source_lang)

# print("Final Response:", final_response)
# print("Translated Response:", translated_response)
# print("Audio Response:", audio_response)

Enter the text"Quels sont les symptômes du diabète?"
Input text is: “What are the symptoms of diabetes?”
Generating response...
You are a highly knowledgeable medical AI assistant. Your task is to provide accurate and helpful information about medical conditions, symptoms, and general health advice. Please answer the following question or respond to the statement:

Question/Statement: “What are the symptoms of diabetes?”

Please provide a detailed and informative response, including relevant medical information, potential symptoms, common treatments, and when to seek professional medical advice. Always emphasize the importance of consulting with a healthcare professional for personalized medical advice and diagnosis.

Response: Diabetes is a chronic condition that affects the body's ability to regulate blood sugar levels. It is characterized by high blood sugar levels due to either insufficient insulin production or resistance to insulin's effects. The most common symptoms of diabetes 

Created audio file: /tmp/tmp2gqsjqpn.mp3
Original (fr)
Translated: “What are the symptoms of diabetes?”
Audio file created: /tmp/tmp2gqsjqpn.mp3


In [ ]:
import transformers
import torch

model_id = "aaditya/OpenBioLLM-Llama3-70B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="auto",
)

messages = [
    {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience."},
    {"role": "user", "content": "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.0,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
https://huggingface.co/ruslanmv/Medical-Llama3-v2
https://huggingface.co/Ashishkr/llama-2-medical-consultation
https://github.com/entbappy/End-to-end-Medical-Chatbot-using-Llama2
https://github.com/rajat709/Llama2-7b-Medical-Model


https://github.com/AI-in-Health/MedLLMsPracticalGuide?tab=readme-ov-file
https://huggingface.co/ProbeMedicalYonseiMAILab/medllama3-v20


https://huggingface.co/Henrychur/MMed-Llama-3-8B